In [ ]:
from SimPEG import Mesh, Utils, mkvc, Maps
import SimPEG.EM.Static.DC as DC
import SimPEG.EM.Static.Utils as StaticUtils
import pylab as plt
import numpy as np
import scipy.sparse as sp
import time
import re
import numpy.matlib as npm
import scipy.interpolate as interpolation
from pymatsolver import PardisoSolver
import os
%pylab 
from scipy.interpolate import griddata


In [ ]:
home_dir = '.'
dsep = '/'

# Specify survey type
# stype = 'dipole-dipole'
stype = 'gradient'
dtype = 'appConductivity'
DOI = False
INVERT = False
# Survey parameters
b = 20 # Tx-Rx seperation
a = 20 # Dipole spacing
n = 15  # Number of Rx per Tx

# Model parameters (background, sphere1, sphere2)
sig = np.r_[1e-2, 1e-1, 1e-3]
eta = 0.1
# Centroid of spheres
loc = np.c_[[-75., 0., -75.], [75., 0., -75.]]
# Radius of spheres
radi = np.r_[50.,50.]

# Inversion parameter
pct = 0.02 # Percent of abs(obs) value
flr = 2e-5 # Minimum floor value
chifact = 100
ref_mod = ([1e-2, 1e-1])

# DOI threshold
cutoff = 0.8
# number of padding cells
padc = 0

# Plotting param
xmin, xmax = -250, 250
ymin, ymax = -150, 150
zmin, zmax = -125, 25
vmin = -2.4771213
vmax = -1.4771213
depth = 200. # Maximum depth to plot
dx_in = 5

#`srvy_end = [(-200.  ,  0.), (200.  ,  0.)]
srvy_end = [(-225.,  0.), (225.,  0.)]
#%% SCRIPT STARTS HERE
nx = int(np.abs(srvy_end[0][0] - srvy_end[1][0]) /dx_in * 1.25)
ny = nx
ny += (ny+1)%2 # Make sure it is odd so the survey is centered

nz = int(np.abs( np.min(loc[2,:]) - np.max(radi) )  /dx_in )

# Create mesh
hxind = [(dx_in,15,-1.3), (dx_in, nx), (dx_in,15,1.3)]
hyind = [(dx_in,15,-1.3), (dx_in, int(ny/3)), (dx_in,15,1.3)]
hzind = [(dx_in,13,-1.3),(dx_in, nz)]

mesh = Mesh.TensorMesh([hxind, hyind, hzind], 'CCN')

# Set background conductivity
model = np.ones(mesh.nC) * sig[0]

In [ ]:
## BLOCK MODEL
# Create blocs for gradient plot
ind = Utils.ModelBuilder.getIndicesBlock(([-25.,-25.,-75]),([25.,25.,-20]),mesh.gridCC)
model[ind] = sig[1]

ind = Utils.ModelBuilder.getIndicesBlock(([-150.,-10.,-25]),([-130.,10.,0]),mesh.gridCC)
model[ind] = sig[1]/2.5
#
ind = Utils.ModelBuilder.getIndicesBlock(([-70.,-10.,-25]),([-50.,10.,0]),mesh.gridCC)
model[ind] = sig[2]
#
ind = Utils.ModelBuilder.getIndicesBlock(([110.,-10.,-25]),([130.,10.,0]),mesh.gridCC)
model[ind] = sig[1]/5.

ind = Utils.ModelBuilder.getIndicesBlock(([-140.,-100.,-25]),([-120.,-80.,0]),mesh.gridCC)
model[ind] = sig[1]/2.5
#
# ind = Utils.ModelBuilder.getIndicesBlock(([-20.,-20.,-25]),([0.,0.,0]),mesh.gridCC)
# model[ind] = sig[1]/2.5
#
ind = Utils.ModelBuilder.getIndicesBlock(([125.,100.,-25]),([145,120.,0]),mesh.gridCC)
model[ind] = sig[1]/5

ind = Utils.ModelBuilder.getIndicesBlock(([80.,-125.,-25]),([100,-105.,0]),mesh.gridCC)
model[ind] = sig[1]/2.5

ind = Utils.ModelBuilder.getIndicesBlock(([-110.,80.,-25]),([-90,100.,0]),mesh.gridCC)
model[ind] = sig[1]/2.5
# Create model Inf
model0 = model.copy()
ind = Utils.ModelBuilder.getIndicesBlock(([-25.,-25.,-75]),([25.,25.,-20]),mesh.gridCC)
model0[ind] = sig[1]*(1-eta)
chg = (model-model0)/model

In [ ]:
plt.figure()
axs = plt.subplot(1,1,1)
# dat1 = mesh.plotSlice(np.log10(model), ind=-5, normal='Z', grid=False, pcolorOpts={'alpha':1}, ax =axs)
dat1 = mesh.plotSlice(chg, ind=-5, normal='Z', grid=False, pcolorOpts={'alpha':0.5}, ax =axs)
axs.set_ylim(ymin,ymax)
axs.set_xlim(xmin,xmax)
plt.gca().set_aspect('equal', adjustable='box')
plt.show()

In [ ]:
def plotPoles(survey,stype,axs):
    for ss in range(survey.nSrc):
        tx = np.c_[survey.srcList[ss].loc]

        if stype == 'dipole-dipole':
            axs.scatter(tx[0,0],tx[0,2],c='k',s=25)

        else:
            axs.scatter(tx[0],tx[2],c='k',s=25)

    tx = np.c_[survey.srcList[0].loc]

    if stype == 'dipole-dipole':
        axs.scatter(tx[0,0],tx[0,2],c='r',s=75, marker='v')
        axs.scatter(tx[0,1],tx[1,2],c='b',s=75, marker='v')
    else:
        axs.scatter(tx[0],tx[2],c='r',s=75, marker='v')

In [ ]:
# Add z coordinate to all survey... assume flat
nz = mesh.vectorNz
var = np.c_[np.asarray(srvy_end),np.ones(2).T*nz[-1]]

# Snap the endpoints to the grid. Easier to create 2D section.
indx = Utils.closestPoints(mesh, var )
endl = np.c_[mesh.gridCC[indx,0],mesh.gridCC[indx,1],np.ones(2).T*nz[-1]]

survey = StaticUtils.gen_DCIPsurvey(endl, mesh, stype, a, b, n)
#survey = DC.SurveyDC.Survey(srcList)

Tx = StaticUtils.getSrc_locs(survey)

dl_len = np.sqrt( np.sum((endl[0,:] - endl[1,:])**2) )
dl_x = ( Tx[-1][0] - Tx[0][0] ) / dl_len
dl_y = ( Tx[-1][1] - Tx[0][1]  ) / dl_len
azm =  np.arctan(dl_y/dl_x)

problem = DC.Problem3D_CC(mesh, sigmaMap = Maps.IdentityMap(mesh))
problem.Solver = PardisoSolver
problem.pair(survey)
dpred = survey.dpred(model)
dpred0 = survey.dpred(model0)

In [ ]:
# Plot stations along line
if stype == 'gradient':
    Rx = survey.srcList[0].rxList[0].locs
    plt.scatter(Tx[0][0::3],Tx[0][1::3],s=40,c='r')
    plt.scatter(np.c_[Rx[0][:,0],Rx[1][:,0]],np.c_[Rx[0][:,1],Rx[1][:,1]],s=20,c='y')
    plt.show()


In [ ]:
dtype = "IP"

if dtype == "DC":
    data = dpred.copy()
elif dtype == "IP":
    data = (dpred0-dpred ) / dpred0    
    
std = abs(data)*pct + flr
survey.dobs = data.copy()
survey.std = std.copy()


Rx = survey.srcList[0].rxList[0].locs
rC1P1 = np.sqrt( np.sum( (npm.repmat(Tx[0][0:2],Rx[0].shape[0], 1) - Rx[0][:,0:2])**2, axis=1 ))
rC2P1 = np.sqrt( np.sum( (npm.repmat(Tx[0][3:5],Rx[0].shape[0], 1) - Rx[0][:,0:2])**2, axis=1 ))
rC1P2 = np.sqrt( np.sum( (npm.repmat(Tx[0][0:2],Rx[0].shape[0], 1) - Rx[1][:,0:2])**2, axis=1 ))
rC2P2 = np.sqrt( np.sum( (npm.repmat(Tx[0][3:5],Rx[0].shape[0], 1) - Rx[1][:,0:2])**2, axis=1 ))

rC1C2 = np.sqrt( np.sum( (npm.repmat(Tx[0][0:2]-Tx[0][3:5],Rx[0].shape[0], 1) )**2, axis=1 ))
rP1P2 = np.sqrt( np.sum( (Rx[0][:,0:2] - Rx[1][:,0:2])**2, axis=1 ))


if dtype == "DC":
    rho = np.abs(data) * 2.*np.pi / ( 1/rC1P1 - 1/rC2P1 - 1/rC1P2 + 1/rC2P2 )
elif dtype == "IP":
    rho = data.copy()



Pmid = (Rx[0][:,0:2] + Rx[1][:,0:2])/2

# Grid points
grid_x, grid_z = np.mgrid[np.min(Rx[0][:,0]):np.max(Rx[1][:,0]):a/10, np.min(Rx[0][:,1]):np.max(Rx[1][:,1]):a/10]
grid_rho = griddata(np.c_[Pmid[:,0],Pmid[:,1]], (abs(rho.T)), (grid_x, grid_z), method='cubic')



#plt.subplot(2,1,2)

def plotdata(data, dtype="DC"):
    if dtype == "DC":
        vmin = -2.4771213
        vmax = -1.4771213
        Model = model.copy()
    elif dtype == "IP":
        vmin = 0
        vmax = 0.1
        Model = 10**(chg)
    fig = plt.figure(figsize = (8,8))
    axs = plt.subplot(2,1,1)
    dat1 = mesh.plotSlice(np.log10(Model), ind=-5, normal='Z', grid=False, pcolorOpts={'alpha':0.5}, ax =axs, clim=(vmin,vmax))
    axs.set_ylim(ymin,ymax)
    axs.set_xlim(xmin,xmax)
    axs.axes.get_xaxis().set_visible(False)
    axs.axes.set_title('')
    plt.gca().set_aspect('equal', adjustable='box')


    # Plot stations along line
    plt.scatter(Tx[0][0::3],Tx[0][1::3],s=40,c='r')
    plt.scatter(np.c_[Rx[0][:,0],Rx[1][:,0]],np.c_[Rx[0][:,1],Rx[1][:,1]],s=10,c='y')
    plt.show()

    if dtype == "DC":
        #plt.tight_layout(pad=0.5)
        axs = plt.subplot(2,1,2)
        im3 = plt.contourf(np.log10(1./grid_rho.T),10, extent = (np.min(grid_x),np.max(grid_x),np.min(grid_z),np.max(grid_z))  ,origin='lower',clim=(vmin,vmax),vmin=vmin,vmax=vmax)
        plt.contour(np.log10(1./grid_rho.T),10, extent = (np.min(grid_x),np.max(grid_x),np.min(grid_z),np.max(grid_z))  ,origin='lower',colors='k')
        
    elif dtype == "IP":
        axs = plt.subplot(2,1,2)
        im3 = plt.contourf((grid_rho.T),10, extent = (np.min(grid_x),np.max(grid_x),np.min(grid_z),np.max(grid_z))  ,origin='lower')
        plt.contour((grid_rho.T),10, extent = (np.min(grid_x),np.max(grid_x),np.min(grid_z),np.max(grid_z))  ,origin='lower',colors='k')
    
    #var = 'Gradient Array - a-spacing: ' + str(a) + ' m'
    #plt.title(var)
    plt.scatter(Tx[0][0::3],Tx[0][1::3],s=40,c='k')
    plt.scatter(np.c_[Rx[0][:,0],Rx[1][:,0]],np.c_[Rx[0][:,1],Rx[1][:,1]],s=10,c='k')
    plt.show()

    axs.set_ylim(ymin,ymax)
    axs.set_xlim(xmin,xmax)
    plt.gca().set_aspect('equal', adjustable='box')
    x = np.linspace(xmin,xmax, 5)
    axs.set_xticks(map(int, x))
    axs.set_xticklabels(map(str, map(int, x)),size=12)
    y = np.linspace(ymin,ymax, 5)
    axs.set_yticks(map(int, y))
    axs.set_yticklabels(map(str, map(int, y)),size=12)

    axs.set_xlabel('x')
    axs.set_ylabel('y')

    pos =  axs.get_position()
    axs.set_position([pos.x0 , pos.y0 +0.05,  pos.width, pos.height])  ## the parameters are the specified position you set
    cbarax = fig.add_axes([pos.x0 + 0.09 , pos.y0 - 0.025,  pos.width*0.75, pos.height*0.05])  ## the parameters are the specified position you set
    if dtype == "DC":
        cbar = fig.colorbar(im3,cax=cbarax, orientation="horizontal", ax = axs, ticks=np.linspace(np.min(np.log10(1./rho)),np.max(np.log10(1./rho)), 3))
        cbar.ax.set_xticklabels([str(int(10**-np.min(np.log10(1./rho)))), str(int(np.round(10**(-np.round(np.min(np.log10(1./rho))+np.max(np.log10(1./rho)))/2)/10)*10)), str(int(round(10**-np.max(np.log10(1./rho)))))])
        cbar.set_label('App. Resistivity ($\Omega \cdot m$)')        
    elif dtype == "IP":
        cbar = fig.colorbar(im3,cax=cbarax, orientation="horizontal", ax = axs, ticks=np.linspace(np.min(rho),np.max(rho), 3))        
        cbar.ax.set_xticklabels(map(str, map(int, np.linspace(np.min(rho),np.max(rho), 3)*1e3)))
        cbar.set_label('App. Chargeability (mV/V)')
plotdata(data, dtype=dtype)        